# scatterizer

> Convert an image into a scatterplot representation

In [ ]:
#| default_exp scatterizer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


class Scatterizer:
    """Converts an input image to a scatterplot representation"""

    rng = np.random.default_rng()
    
    @staticmethod
    def _convert_arr_to_int8(arr):
        ratio = np.amax(arr) / 255
        return (arr / ratio).astype('uint8')

    @classmethod
    def load_img(cls, img_file):
        img = np.asarray(Image.open(img_file).convert('RGB'))
        assert img.ndim == 3
        img_uint8 = cls._convert_arr_to_int8(img)
        
        img_gray = np.asarray(Image.open(img_file).convert('L'))
        img_gray_uint8 = cls._convert_arr_to_int8(img_gray)
        return img_uint8, img_gray_uint8

    @staticmethod
    def binarize(img_gray_uint8):
        _, img_bin = cv2.threshold(img_gray_uint8, 0, 1, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        return img_bin

    @classmethod
    def sample_hot_pixels(cls, img_bin, n_samples=1000):
        coords = np.argwhere(img_bin)
        sample_coords = cls.rng.choice(coords, size=n_samples, replace=False)
        return sample_coords

    @classmethod
    def generate_scatter(cls, img, sample_coords, var=0.1):
        # opencv flips x and y
        y, x, z = img.shape
        smaller_dim_size = min(x, y)
        stdev_real = np.sqrt(smaller_dim_size * var)

        # sampling from many multivariate normals is not vectorized in numpy
        # so instead sample from 2 univariate normals and recombine
        x_sample_coords, y_sample_coords = sample_coords[:, 1], sample_coords[:, 0]
        x_coords = cls.rng.normal(x_sample_coords, stdev_real).astype('int')
        y_coords = cls.rng.normal(y_sample_coords, stdev_real).astype('int')
        keep_coord_rows = (x_coords < x) & (y_coords < y)
        scatter_coords = np.stack([y_coords, x_coords], axis=1)
        scatter_coords = scatter_coords[keep_coord_rows]
        
        colors = img[y_sample_coords, x_sample_coords, :]
        colors = colors[keep_coord_rows]
        return scatter_coords, colors

    @staticmethod
    def plot_scatter(img, scatter_coords, colors, alpha=0.25, width=6, pointsz=25, pngpath=None):
        # opencv flips x and y
        y, x, z = img.shape
        height = width * y / x
        x_coords, y_coords = scatter_coords[:, 1], scatter_coords[:, 0]
        # make rgba values for each point
        rgbs = colors / 255
        rgb_alphas = np.append(rgbs, alpha*np.ones((colors.shape[0], 1)), axis=1)
        rgb_half_alphas = np.append(rgbs, alpha/3*np.ones((colors.shape[0], 1)), axis=1)
        # plot
        fig = plt.figure(figsize=(width, height))
        plt.scatter(
            x_coords,
            y - y_coords,  # plt.scatter assumes bottom->up direction on y-axis
            s=pointsz,
            marker='o',
            edgecolors=rgb_alphas,
            facecolors=rgb_half_alphas
        )
        # add invisible points for the far corners
        plt.scatter([0, x], [0, y], c=[(1, 1, 1, 0)]*2)
        # tidy up
        plt.axis('off')
        if pngpath:
            plt.savefig(pngpath, bbox_inches='tight')
        plt.show()
    
    @classmethod
    def scatter(
        cls,
        img_file:str, # image filepath
        n_samples:int=1000, # number of points to sample
        var:float=0.1, # amount of scatter (between 0 and 1 usually best)
        alpha:float=0.25, # opacity of each point (0 to 1)
        width:float=None, # can specify width (in) of output img, defaults to input img
        pointsz:int=25, # size of each point
        savefig:bool=True # whether to save the output img
    ):
        """Executes the full pipeline to create the output image"""
        
        img, img_gray = cls.load_img(img_file)
        img_bin = cls.binarize(img_gray)
        sample_coords = cls.sample_hot_pixels(img_bin, n_samples=n_samples)
        
        if width is not None:
            y, x = img_gray.shape
            plt.figure(figsize=(width, width*y/x))            
            
        plt.imshow(img)
        plt.axis('off')
        figsize = plt.gcf().get_size_inches()
        plt.show()
        
        scatter_coords, colors = cls.generate_scatter(img, sample_coords, var=var)
        
        pngpath = None
        if savefig:
            filepath_noext = Path(img_file).with_suffix("")
            pngpath = f"{filepath_noext}_scatterized.png"
            
        cls.plot_scatter(
            img,
            scatter_coords,
            colors,
            alpha=alpha,
            width=figsize[0],
            pointsz=pointsz,
            pngpath=pngpath
        )

In [ ]:
show_doc(Scatterizer.scatter)

---

[source](https://github.com/davegrays/logo-dataizer/blob/main/logo_dataizer/scatterizer.py#L106){target="_blank" style="float:right; font-size:smaller"}

### Scatterizer.scatter

>      Scatterizer.scatter (img_file:str, n_samples:int=1000, var:float=0.1,
>                           alpha:float=0.25, width:float=None, pointsz:int=25,
>                           savefig:bool=True)

Executes the full pipeline to create the output image

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| img_file | str |  | image filepath |
| n_samples | int | 1000 | number of points to sample |
| var | float | 0.1 | amount of scatter (between 0 and 1 usually best) |
| alpha | float | 0.25 | opacity of each point (0 to 1) |
| width | float | None | can specify width (in) of output img, defaults to input img |
| pointsz | int | 25 | size of each point |
| savefig | bool | True | whether to save the output img |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()